## Before Refactoring

In [ ]:
class FoodDeliveryApp:
    def __init__(self):
        self.orders = []
        self.drivers = ['Alice', 'Bob', 'Charlie']

    def place_order(self, customer_name, food_item, payment_method):
        # Process order and payment
        print(f"Processing order for {customer_name}: {food_item}")
        if payment_method == 'card':
            print("Processing card payment...")
        elif payment_method == 'cash':
            print("Cash on delivery selected.")
        else:
            print("Invalid payment method!")
            return

        self.orders.append((customer_name, food_item))
        print(f"Order placed successfully for {customer_name}.")

        # Assign delivery driver
        assigned_driver = self.drivers[0]  # Always assigns the first driver
        print(f"{assigned_driver} will deliver the order.")

        # Send notification
        print(f"Notification sent to {customer_name}: Your order is on the way!")

# Test the application
app = FoodDeliveryApp()
app.place_order("John Doe", "Pizza", "card")

## After Refactoring

In [12]:
class Order:
    def __init__(self, customer_name, food_item):
        self.customer_name = customer_name
        self.food_item = food_item

class Payment:
    def pay(self, order: Order):
        raise NotImplementedError()

class PayWithCard(Payment):
    def pay(self, order: Order):
        print(f"Processing credit card payment for {order.customer_name}...")

class PayWithCash(Payment):
    def pay(self, order: Order):
        print(f"Processing cash payment for {order.customer_name}...")

class DeliveryMethod:
    def assign(self, order: Order):
        raise NotImplementedError()

class DriverAssignment(DeliveryMethod):
    def __init__(self):
        self.drivers = ['Alice', 'Bob', 'Charlie']
    def assign(self, order: Order):
        assigned_driver = self.drivers[0]
        print(f"{assigned_driver} is assigned to deliver {order.food_item}")
        return assigned_driver

class Notifier:
    def send(self, order: Order):
        raise NotImplementedError()

class SimpleNotifier(Notifier):
    def send(self, order: Order):
        print(f"Mr. {order.customer_name}! Your {order.food_item} is on the way!")


class FoodDeliveryApp:
    def __init__(self, payment: Payment, assigner: DeliveryMethod, notifier: Notifier):
        self.payment = payment
        self.assigner = assigner
        self.notifier = notifier

    def place_order(self, order: Order):
        self.payment.pay(order)
        self.assigner.assign(order)
        self.notifier.send(order)

# Test
order = Order("Tanaka", "Ramen")
app = FoodDeliveryApp(
    PayWithCard(),
    DriverAssignment(),
    SimpleNotifier()
)
app.place_order(order)

Processing credit card payment for Tanaka...
Alice is assigned to deliver Ramen
Mr. Tanaka! Your Ramen is on the way!


## Model Answer

In [11]:
# Applying SRP: Separate concerns into individual classes
class Order:
    def __init__(self, customer_name, food_item):
        self.customer_name = customer_name
        self.food_item = food_item

# Applying OCP & DIP: Using abstraction for payment methods
class PaymentProcessor:
    def process_payment(self, payment_method):
        raise NotImplementedError("Subclasses must implement process_payment method")

class CardPayment(PaymentProcessor):
    def process_payment(self):
        print("Processing card payment...")
        return True

class CashPayment(PaymentProcessor):
    def process_payment(self):
        print("Cash on delivery selected.")
        return True

# Applying LSP: Abstracting delivery types
class DeliveryMethod:
    def assign_driver(self):
        raise NotImplementedError("Subclasses must implement assign_driver method")

class DriverDelivery(DeliveryMethod):
    def __init__(self):
        self.drivers = ['Alice', 'Bob', 'Charlie']

    def assign_driver(self):
        assigned_driver = self.drivers[0]  # Placeholder assignment logic
        print(f"{assigned_driver} will deliver the order.")
        return assigned_driver

# Applying SRP: Separate Notification Service
class NotificationService:
    def send_notification(self, customer_name):
        print(f"Notification sent to {customer_name}: Your order is on the way!")

# The main application class follows DIP by depending on abstractions
class FoodDeliveryApp:
    def __init__(self):
        self.orders = []
        self.notification_service = NotificationService()

    def place_order(self, customer_name, food_item, payment_method):
        order = Order(customer_name, food_item)
        payment_processor = CardPayment() if payment_method == 'card' else CashPayment()

        if not payment_processor.process_payment():
            return

        self.orders.append(order)
        print(f"Order placed successfully for {customer_name}.")

        delivery = DriverDelivery()
        assigned_driver = delivery.assign_driver()
        self.notification_service.send_notification(customer_name)

# Test the improved application
app = FoodDeliveryApp()
app.place_order("John Doe", "Pizza", "card")

Processing card payment...
Order placed successfully for John Doe.
Alice will deliver the order.
Notification sent to John Doe: Your order is on the way!
